In [2]:
import keras
import tensorflow as tf
import json, os, time, copy
import numpy as np
import skimage
import seaborn as sns
import glob
import SimpleITK as sitk

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D 
from keras import backend as K
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import imageio
sns.set()

Using TensorFlow backend.


In [3]:
import cv2

import numpy as np

# Get array
#rootdir = "/Users/hkim20/Documents/Unlabeled_predictions/Glucose/"
#outdir = "/Users/hkim20/Downloads/"
rootdir = "D:/INS-1E Source Images/Unlabeled_train"
niiPaths = []
fill_value = 0
num_files=188
imageNewArray = np.zeros((num_files*128,128,128,3))   

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file.endswith("nii.gz")):
                niiPaths.append(os.path.join(subdir, file))

print("Number of source filePaths: " + str(len(niiPaths)))
#print(niiPaths)

allImages = np.zeros((num_files*128, 128, 128))

for fileCount, filePath in enumerate(niiPaths[:]):
    
    image = sitk.ReadImage(filePath) 
   # im = sitk.GetImageFromArray(image)
    
    print("Processing image: " + str(fileCount + 1) + ". Filename: "+ filePath)

    num_dim = image.GetDimension()
    
    orig_pixelid = image.GetPixelIDValue()
    orig_origin = image.GetOrigin()
    orig_direction = image.GetDirection()
    orig_spacing = np.array(image.GetSpacing())
    orig_size = np.array(image.GetSize(), dtype=np.int)

    
    
    new_size = [128,128,128] #  SimpleITK expects lists, not ndarrays

    new_spacing = [sz*spc/nsz for nsz,sz,spc in zip(new_size, image.GetSize(), image.GetSpacing())]

    resample_filter = sitk.ResampleImageFilter()

    resampled_sitk_image = resample_filter.Execute(image,
                                                   new_size,
                                                   sitk.Transform(),
                                                   sitk.sitkNearestNeighbor,
                                                   orig_origin,
                                                   new_spacing,
                                                   orig_direction,
                                                   fill_value,
                                                   orig_pixelid)
   
   # resampled_sitk_image = np.expand_dims (resampled_sitk_image, axis =2)

    

    imageArray = sitk.GetArrayFromImage(resampled_sitk_image)
    
    for j in range(64):
        allImages[64*fileCount + j] = imageArray[32+j] # middle slides    
    
print ("done uploading")

for i in range(num_files*128):
        for j in range (128):
            for k in range (128):
                for l in (0,1,2):
                    imageNewArray[i][j][k][l] = allImages[i][j][k]

#imageNewArray = np.expand_dims(imageNewArray, axis=3)
print(imageNewArray.shape)


Number of source filePaths: 188
Processing image: 1. Filename: D:/INS-1E Source Images/Unlabeled_train\25mM Glucose\1m\Stevens_pancreatic_INS_1E_1_819_10_pre_rec.nii.gz
Processing image: 2. Filename: D:/INS-1E Source Images/Unlabeled_train\25mM Glucose\1m\Stevens_pancreatic_INS_1E_1_819_11_pre_rec.nii.gz
Processing image: 3. Filename: D:/INS-1E Source Images/Unlabeled_train\25mM Glucose\1m\Stevens_pancreatic_INS_1E_1_819_12_pre_rec.nii.gz
Processing image: 4. Filename: D:/INS-1E Source Images/Unlabeled_train\25mM Glucose\1m\Stevens_pancreatic_INS_1E_1_819_2_pre_rec.nii.gz
Processing image: 5. Filename: D:/INS-1E Source Images/Unlabeled_train\25mM Glucose\1m\Stevens_pancreatic_INS_1E_1_819_3_pre_rec.nii.gz
Processing image: 6. Filename: D:/INS-1E Source Images/Unlabeled_train\25mM Glucose\1m\Stevens_pancreatic_INS_1E_1_819_4_pre_rec.nii.gz
Processing image: 7. Filename: D:/INS-1E Source Images/Unlabeled_train\25mM Glucose\1m\Stevens_pancreatic_INS_1E_1_819_5_pre_rec.nii.gz
Processing im

In [4]:
##@USC
labelArray = np.zeros((num_files*128, 7))
print(len(niiPaths))
for fileCount, filePath in enumerate(niiPaths[:]):
    label=np.zeros((7))
    if "No treatment" in filePath:
        label[0] = 1
    elif "Ex" in filePath:
        if "1m\\" in filePath:
            label[1] = 1
        elif "5m\\" in filePath:
            label[2] = 1
        elif "30m\\" in filePath:
            label[3] = 1
    else:
        if "1m\\" in filePath:
            label[4] = 1
        elif "5m\\" in filePath:
            label[5] = 1
        elif "30m\\" in filePath:
            label[6] = 1
    for j in range(128):
        labelArray[128*fileCount + j] = label
    
    #print("Saving Labels: " + str(fileCount+1) + ". Label:" + str(label) + "for filepath " + str(filePath))
    
print(np.unique(labelArray))

188
[0. 1.]


In [ ]:
###FOR TESTING###

# Get array
test_rootdir = "D:/INS-1E Source Images/Unlabeled_test"
test_niiPaths = []
fill_value = 0
num_files=223


test_imageNewArray = np.zeros((num_files*128,128,128,3))   


for subdir, dirs, files in os.walk(test_rootdir):
    for file in files:
        if(file.endswith("nii.gz")):
                test_niiPaths.append(os.path.join(subdir, file))

print("Number of source filePaths: " + str(len(test_niiPaths)))

test_allImages = np.zeros((num_files*128, 128, 128))

for fileCount, filePath in enumerate(test_niiPaths[:]):
    
    image = sitk.ReadImage(filePath) 
   # im = sitk.GetImageFromArray(image)
    
    print("Processing image: " + str(fileCount + 1) + ". Filename: "+ filePath)

    num_dim = image.GetDimension()
    
    orig_pixelid = image.GetPixelIDValue()
    orig_origin = image.GetOrigin()
    orig_direction = image.GetDirection()
    orig_spacing = np.array(image.GetSpacing())
    orig_size = np.array(image.GetSize(), dtype=np.int)

    new_size = [128,128,128] #  SimpleITK expects lists, not ndarrays

    new_spacing = [sz*spc/nsz for nsz,sz,spc in zip(new_size, image.GetSize(), image.GetSpacing())]

    resample_filter = sitk.ResampleImageFilter()

    resampled_sitk_image = resample_filter.Execute(image,
                                                   new_size,
                                                   sitk.Transform(),
                                                   sitk.sitkNearestNeighbor,
                                                   orig_origin,
                                                   new_spacing,
                                                   orig_direction,
                                                   fill_value,
                                                   orig_pixelid)

    test_imageArray = sitk.GetArrayFromImage(resampled_sitk_image)
    
    for j in range(64):
        test_allImages[64*fileCount + j] = test_imageArray[32+j] # middle slides

print ("done uploading")


for i in range(num_files*128):
        for j in range (128):
            for k in range (128):
                for l in (0,1,2):
                    test_imageNewArray[i][j][k][l] = test_allImages[i][j][k]

#imageNewArray = np.expand_dims(imageNewArray, axis=3)
print(test_imageNewArray.shape)

Number of source filePaths: 223
Processing image: 1. Filename: D:/INS-1E Source Images/Unlabeled_test\25mM Glucose\1m\Stevens_pancreatic_INS_1E_1_819_13_pre_rec.nii.gz
Processing image: 2. Filename: D:/INS-1E Source Images/Unlabeled_test\25mM Glucose\1m\Stevens_pancreatic_INS_1E_1_819_1_pre_rec.nii.gz
Processing image: 3. Filename: D:/INS-1E Source Images/Unlabeled_test\25mM Glucose\1m\Stevens_pancreatic_INS_1E_1_819_7_pre_rec.nii.gz
Processing image: 4. Filename: D:/INS-1E Source Images/Unlabeled_test\25mM Glucose\1m\Stevens_pancreatic_INS_1E_5_817_1_pre_rec.nii.gz
Processing image: 5. Filename: D:/INS-1E Source Images/Unlabeled_test\25mM Glucose\1m\Stevens_pancreatic_INS_1E_5_817_4_pre_rec.nii.gz
Processing image: 6. Filename: D:/INS-1E Source Images/Unlabeled_test\25mM Glucose\1m\Stevens_pancreatic_INS_1E_5_817_8_pre_rec.nii.gz
Processing image: 7. Filename: D:/INS-1E Source Images/Unlabeled_test\25mM Glucose\1m\Stevens_pancreatic_INS_25Glu_1min_930_5_pre_rec.nii.gz
Processing image

In [1]:
test_labelArray = np.zeros((num_files*64, 7))
for fileCount, filePath in enumerate(test_niiPaths[:]):
    label = np.zeros((7))
    if "No treatment" in filePath:
        label[0] = 1
    elif "Ex" in filePath:
        if "1m\\" in filePath:
            label[1] = 1
        elif "5m\\" in filePath:
            label[2] = 1
        elif "30m\\" in filePath:
            label[3] = 1
    else:
        if "1m\\" in filePath:
            label[4] = 1
        elif "5m\\" in filePath:
            label[5] = 1
        elif "30m\\" in filePath:
            label[6] = 1
    for j in range(64):
        test_labelArray[64*fileCount + j] = label
    
    #print("Saving Labels: " + str(fileCount+1) + ". Label:" + str(label) + "for filepath " + str(filePath))
  
print(len(test_labelArray))
print(np.unique(test_labelArray))

NameError: name 'np' is not defined

In [24]:
batch_size = 64
num_classes = 7
num_labels = 5
epochs = 12

# input image dimension

img_rows, img_cols = 128, 128

# the data, split between train and test sets

x_train = imageNewArray
y_train = labelArray

x_test = test_imageNewArray
y_test= test_labelArray

input_shape = (img_rows, img_cols,3)

In [25]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(np.unique(y_train))
print(np.unique(y_test))
print(y_test[0])

(37120, 128, 128, 3)
(37120, 7)
(12032, 128, 128, 3)
(12032, 7)
[0.]
[0. 1.]
[0. 0. 1. 0. 0. 0. 0.]


In [35]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

img_width, img_height = 128, 128
train_data_dir = "D:/INS-1E Source Images/Unlabeled_train"
validation_data_dir = "D:/INS-1E Source Images/Unlabeled_test"

batch_size = 16
epochs = 10

model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

In [27]:
for layer in model.layers[:5]:
    layer.trainable = False
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(7, activation="softmax")(x)

In [28]:
model_final = Model(input = model.input, output = predictions)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [29]:
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [30]:
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

In [31]:
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

In [ ]:
model_final.fit(x_train, y_train,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks = [checkpoint, early])
          
score = model_final.evaluate(x_test, y_test, verbose=1)

Train on 37120 samples, validate on 12032 samples
Epoch 1/10
37120/37120 [==============================] - 18213s 491ms/step - loss: 0.0000e+00 - acc: 0.4674 - val_loss: 2.1778 - val_acc: 0.0323

Epoch 00001: val_acc improved from -inf to 0.03233, saving model to vgg16_1.h5
Epoch 2/10
37120/37120 [==============================] - 71566s 2s/step - loss: 0.0000e+00 - acc: 0.4659 - val_loss: 2.1778 - val_acc: 0.0323

Epoch 00002: val_acc did not improve from 0.03233
Epoch 3/10
37120/37120 [==============================] - 16767s 452ms/step - loss: 0.0000e+00 - acc: 0.4731 - val_loss: 2.1778 - val_acc: 0.0323

Epoch 00003: val_acc did not improve from 0.03233
Epoch 4/10
18144/37120 [=============>................] - ETA: 12:54:16 - loss: 0.0000e+00 - acc: 0.4660

In [ ]:

model.save_weights('D:/INS-1E Source Images/model_weights_01_09_2020.h5') 
print("saved")

with open('D:/INS-1E Source Images/model_architecture_01_09_2020.json', 'w') as f:
    f.write(model.to_json())

In [ ]:
from keras.models import model_from_json

# Model reconstruction from JSON file
with open('D:/INS-1E Source Images/model_architecture_01_09_2020.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('D:/INS-1E Source Images/model_weights_01_09_2020.h5') #change names


In [ ]:
# confusion matrix
from sklearn.metrics import confusion_matrix

test_labels = np.zeros((len(test_labelArray)))
for i,label in enumerate(test_labelArray):
    for j in range(7):
        if label[j] == 1:
            test_labels[i] = j

print(ynew)
print(test_labels)
confusionArray = confusion_matrix(test_labels, ynew)
print(confusionArray)

In [ ]:
print(confusionArray.shape)
confusionFloat = np.zeros((7,7))
for i in range(7):
    sum = 0
    for j in range(7):
        sum = sum + confusionArray[i][j]
    print(sum)
    for k in range(7):
        confusionFloat[i][k] = round(confusionArray[i][k]/sum, 2)

print(confusionFloat)
img = plt.imshow(confusionFloat)
img.set_cmap('hot')
plt.axis('off')
plt.show()